In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/project/

In [ ]:
%pip install -e .

In [ ]:
%cd elpv/src

In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/project/elpv')
sys.path

In [1]:
from utils.classifier import *
from utils.dataloader import *
from utils.features import *
from utils.helper import *
from utils.processing import *
from functools import partial
import pickle

In [2]:
RANDOM_STATE = 66
ROOT_PATH, DATA_PATH, OUT_PATH = get_paths()
print(ROOT_PATH)

/Users/moooooo16/Documents/Computer Vision/Project/elpv


In [3]:
img_path, prob, types, labels= load_data(DATA_PATH)
print(len(img_path), 
      len(prob),
      len(types),
      len(labels))

[0 1 2 3 4 5 6 7] [588 117  56 313 920 178  50 402]
2624 2624 2624 2624


# Class mapping 

|Label | prob | type|
|------|------|-----|
|0     | 0.0  | mono|
|1     | 0.333| mono|
|2     | 0.667| mono|
|3     | 1.0  | mono|
|4     | 0.0  | poly|
|5     | 0.333| poly|
|6     | 0.667| poly|
|7     | 1.0  | poly|

In [4]:
from sklearn.model_selection import train_test_split

feature_builder = FeatureExtraction(DATA_PATH, img_path, labels)
X_train, X_test, y_train, y_test = feature_builder.split_data(train_test_split, randome_state=RANDOM_STATE, stratify=True, split_ratio=0.25)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1968, 300, 300) (1968,)
(656, 300, 300) (656,)


In [5]:
X_train_aug, y_train_aug = feature_builder.augmentation(X_train, y_train, augment_funcs=[flip_x, flip_y, rotate])
print(X_train_aug.shape, y_train_aug.shape)
print(X_test.shape, y_test.shape)

Augmenting images: 100%|██████████| 1968/1968 [00:00<00:00, 111742.12it/s]

Augmenting done, added image: 900
(2868, 300, 300) (2868,)
(656, 300, 300) (656,)


In [6]:
clach_params = partial(clach_img, clipLimit=2.0, tileGridSize=(8, 8))
gaussian_params = partial(guassian_blur, kernel_size=(0,0), sigmaX=3)
lap_params = partial(lap_feature, dst = -1, ksize = 5)
kernel = np.ones((2,2), np.uint8)
morpo_opening_params = partial(morpo_opening, kernel=kernel, iterations=1)


X_train_prepro = feature_builder.preprocess(X_train_aug, preprocess_funcs=[clach_params, gaussian_params, lap_params,min_max_normalize, morpo_opening_params])
X_test_prepro = feature_builder.preprocess(X_test, preprocess_funcs=[clach_params, gaussian_params, lap_params,min_max_normalize, morpo_opening_params])
print(X_train_prepro.shape, y_train_aug.shape)
print(X_test_prepro.shape, y_test.shape)

Pre-processing images:   0%|          | 0/2868 [00:00<?, ?it/s]

After clach_img: range 11 to 236
After guassian_blur: range 24 to 221
After lap_feature: range 0 to 10
After min_max_normalize: range 0 to 255
After morpo_opening: range 0 to 178


Pre-processing images: 100%|██████████| 656/656 [00:00<00:00, 1820.99it/s]

(2868, 300, 300) (2868,)
(656, 300, 300) (656,)


In [7]:
from sklearn.cluster import KMeans

train_kps, train_sift_descriptors, train_empty_desc = feature_builder.get_sift_descriptor(X_train_prepro)
test_kps, test_sift_descriptors, test_empty_desc = feature_builder.get_sift_descriptor(X_test_prepro)

if len(train_empty_desc) > 0:
    print("Empty descriptor: ", train_empty_desc)
if len(test_empty_desc) > 0:
    print("Empty descriptor: ", test_empty_desc)


Calculating descriptors: 100%|██████████| 656/656 [00:05<00:00, 125.77it/s]


In [8]:
ks = [80, 160, 240, 320, 400]

kmeans, train_sift_hists = feature_builder.build_sift_cluster(KMeans, train_sift_descriptors, ks, state=RANDOM_STATE)


test_sift_hists = {}
for k in ks:
    test_sift_hist = feature_builder.get_hist(test_sift_descriptors, kmeans[k], k)
    test_sift_hists[k] = test_sift_hist

print(train_sift_hists[ks[0]].shape)
print(test_sift_hists[ks[0]].shape) 

Calculating kmeans for k = 80


Building histogram for k = 80: 100%|██████████| 2868/2868 [00:04<00:00, 645.68it/s]


Calculating kmeans for k = 160


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/torch/lib/python3.11/site-packages/numpy/core/multiarray.py", line 346, in where
    @array_function_from_c_func_and_dispatcher(_multiarray_umath.where)
    
KeyboardInterrupt: 


In [ ]:
train_hog_des = feature_builder.get_hog_features(X_train_prepro, orient=9, 
                                                 pix_per_cell=25, cell_per_block=2, 
                                                 vis=False, block_norm='L2-Hys')

test_hog_des = feature_builder.get_hog_features(X_test_prepro, orient=9,
                                                pix_per_cell=25, cell_per_block=2,
                                                vis=False, block_norm='L2-Hys')

print(train_hog_des.shape)
print(test_hog_des.shape)

In [ ]:
with open(os.path.join(OUT_PATH, 'sift_X.pkl'), 'wb') as f:
    pickle.dump((train_sift_hists, test_sift_hists), f)
    
with open(os.path.join(OUT_PATH, 'y.pkl'), 'wb') as f:
    pickle.dump((y_train_aug, y_test), f)
    
with open(os.path.join(OUT_PATH, 'hog_X.pkl'), 'wb') as f:
    pickle.dump((train_hog_des, test_hog_des), f)

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

param = {
    'kernel': ['rbf'],
    'C' : [0.01, 0.1, 1, 10, 100],
    'class_weight': ['balanced']
}


clf_collections = {}
predictions_collections = {}
distance_collections = {}



In [ ]:
logger = my_logger(os.path.join(OUT_PATH, 'logs/sift_svm.log'), 'sift_svm')
logger.warning(f'k,i,j,best_score,,best_clf,mean_train_score,mean_test_score')




for k in ks:
    print("K: ", k)
    total_classes = np.unique(y_train)
    train_X = StandardScaler.fit_transform(train_sift_hists[k])
    test_X = StandardScaler.fit_transform(test_sift_hists[k])
    
    
    best_clfs = one_vs_other_up_sampling(X_train = train_X, y_train=y_train_aug, 
                                         total_classes= total_classes, clf = SVC, param = param, 
                                         k = k, logger = logger, 
                                         knn= KNeighborsClassifier, n_neighbors=5, verbose=3)

    distance , pred = distance_vote(test_X, best_clfs)
    acc, conf_mat = get_report(y_test, pred)
    print()
    print(f'Overall Test Accuracy: {acc:.2f}')
    print(f'Confusion Matrix: \n{conf_mat}')
    print('-'*100)
    
    clf_collections[k] = best_clfs
    predictions_collections[k] = pred
    distance_collections[k] = distance

In [ ]:
logger = my_logger(os.path.join(OUT_PATH, 'logs/hog_svm.log'), 'hogsvm')
logger.warning(f'i,j,best_score,,best_clf,mean_train_score,mean_test_score')


train_X = StandardScaler.fit_transform(train_hog_des)
test_X = StandardScaler.fit_transform(test_hog_des)


total_classes = np.unique(y_train)
k = None


clfs = one_vs_other_up_sampling(total_classes, train_X, y_train_aug,
                            clf = SVC,
                            param=param, k=k, logger=logger,
                            knn = KNeighborsClassifier, n_neighbors=5)



distance , pred = distance_vote(test_X, clfs, None)

acc, conf_mat = get_report(y_test, pred)
print()
print(f'Overall Test Accuracy: {acc:.2f}')
print(f'Confusion Matrix: \n{conf_mat}')
print('-'*100)